# NMR vs EPR vs muSR

Visualizing differences in ZFS signatures for different techniques (NMR, EPR, muSR) in transverse field (TF) experiments using S=1/2, I=1/2 system

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import plotly.colors as pc
import xarray as xr
import netCDF4 as nc
# from ALC_simulations.utils import stick_spectrum
from scipy.io import loadmat, savemat

import Python.plot_settings
pio.templates.default = "DemonLab"


In [ ]:
import plotly.graph_objects as go

def add_doubleheaded_arrow(
    fig,
    xrange,
    yrange,
    x1,
    x2=None,
    y=None,
    label="",
    y2=None,
    orientation="h",
    label_offset=-2,
    font_size=20,
    line_width=2,
    head_size=14,
    arrow_indent=0.02,
):
    """
    Adds a horizontal or vertical double-headed arrow with a label to a Plotly figure,
    using triangle markers for arrowheads.

    Parameters
    ----------
    fig : plotly.graph_objects.Figure
        Target figure.
    range : tuple
    Range of arrow points.
    x1, x2 : float
        For horizontal arrows: start and end x positions.
        For vertical arrows: x1 is the x position (x2 ignored).
    y, y2 : float
        For vertical arrows: start and end y positions.
        For horizontal arrows: y is given in paper coordinates (0–1).
    orientation : {"h", "v"}
        Direction of the arrow.
    label : str
        Text label to display at the midpoint.
    label_offset : float
        Offset of the label in pixels (vertical for horizontal arrows, horizontal for vertical).
    font_size : int
        Label font size.
    line_width : float
        Width of the arrow shaft.
    head_size : float
        Size of the triangle arrowheads.
    """

    if orientation == "h":
        if x2 is None or y is None:
            raise ValueError("x2 and y must be provided for horizontal arrows.")

        y = y * abs(yrange[1] - yrange[0]) + yrange[0]

        label_shift = label_offset * abs(yrange[1] - yrange[0])
        arrow_shift = arrow_indent * abs(xrange[1] - xrange[0])

        # Shaft
        fig.add_shape(
            type="line",
            x0=x1+arrow_shift, y0=y,
            x1=x2-arrow_shift, y1=y,
            line=dict(color="black", width=line_width),
            xref="x", yref='y',
            showlegend=False,
        )

        # Arrowheads
        fig.add_trace(go.Scatter(
            x=[x1+arrow_shift, x2-arrow_shift],
            y=[y, y],
            mode="markers",
            marker_symbol=["triangle-left", "triangle-right"],
            marker_size=head_size,
            marker_color="black",
            marker_line_color="black",
            marker_line_width=1,
            hoverinfo="skip",
            xaxis="x",
            yaxis="y",
            showlegend=False,
        ))

        # Label
        fig.add_annotation(
            x=(x1 + x2) / 2,
            y=y+label_shift,
            text=label,
            showarrow=False,
            font=dict(size=font_size),
            xref="x",
            yref="y",
        )

    elif orientation == "v":
        if y2 is None or x1 is None:
            raise ValueError("x1 and y2 must be provided for vertical arrows.")

        x = x1 * abs(xrange[1] - xrange[0]) + xrange[0]

        label_shift = label_offset * abs(xrange[1] - xrange[0]) * 1.5
        arrow_shift = arrow_indent * abs(yrange[1] - yrange[0]) / 1.5  # divide by 1.5 as figures have size 600x400

        # Shaft
        fig.add_shape(
            type="line",
            x0=x, y0=y+arrow_shift,
            x1=x, y1=y2-arrow_shift,
            line=dict(color="black", width=line_width),
            xref="x", yref="y",
            showlegend=False,
        )

        # Arrowheads
        fig.add_trace(go.Scatter(
            x=[x, x],
            y=[y+arrow_shift, y2-arrow_shift],
            mode="markers",
            marker_symbol=["triangle-down", "triangle-up"],
            marker_size=head_size,
            marker_color="black",
            marker_line_color="black",
            marker_line_width=1,
            hoverinfo="skip",
            showlegend=False,
        ))

        # Label
        fig.add_annotation(
            x=x+label_shift,
            y=(y + y2) / 2,
            text=label,
            showarrow=False,
            font=dict(size=font_size),
            xref="x",
        )

    else:
        raise ValueError("orientation must be either 'h' or 'v'.")


def get_concoluted_spectrum(freq, amps, freq_axis, gamma):
    """
    freq: array of peak frequencies
    amps: array of peak amplitudes
    freq_axis: frequency axis for plotting
    gamma: FWHM of each Lorentzian peak
    """
    def lorentzian(x, x0, amp, gamma):
        return amp * (0.5*gamma)**2 / ((x - x0)**2 + (0.5*gamma)**2)

    spectrum = np.zeros_like(freq_axis)
    for f, a in zip(freq, amps):
        if np.isfinite(f) and np.isfinite(a):
            spectrum += lorentzian(freq_axis, f, a, gamma)
    return spectrum

# Problem of MathJax rendering, it can't go beyond font size 20
# therefore use r'$\Large ...$' for larger symbols
def create_ZF_base_figure():
    fig = go.Figure()
    fig.update_layout(
        xaxis_title='𝜈',    # or use r'$\Large \nu$'
        yaxis_title='Amplitude',
        xaxis=dict(
            showgrid=False,
            tickvals=[0],
            ticktext=['0'],
            # title_standoff=8,        # decrease distance between x label and axis
            minor=dict(ticks='', showgrid=False),  # remove minor ticks and minor grid
            ticklabelstandoff=-8,
        ),
        margin=dict(l=90, r=20, t=20, b=55),
        height=400, width=600,
        template='DemonLab',
    )
    return fig


## Zero Field Splitting

Scaling constants used for all ZF figures


### NMR

#### Only dipolar coupling of D_parallel = 2 MHz
expect 2 peaks with a splitting of 3D

In [ ]:
xarr = xr.load_dataset('ALC_simulations/Data/TF_NMR_dipolar_cc.nc')
theta = 0
B = 0

freq = xarr['frequencies'].sel(B=B, theta=theta).values * 1e3 # Convert to GHz
amps = xarr['amplitudes'].sel(B=B, theta=theta).values

# Define the frequency axis for plotting
freq_min, freq_max = -2.5, 2.5
n_points = 8000
freq_axis = np.linspace(freq_min, freq_max, n_points)
gamma = 0.01  # MHz

spectrum = get_concoluted_spectrum(freq, amps, freq_axis, gamma)

print(max(spectrum))

xrange = [freq_min, freq_max]
yrange = [-0.015, 0.56]

# Plot
fig = create_ZF_base_figure()
fig.add_trace(go.Scatter(x=freq_axis, y=spectrum, mode='lines', line=dict(color='black', width=2), showlegend=False))
add_doubleheaded_arrow(fig,
                       xrange, yrange, x1=freq[0], x2=freq[1], y=0.75,
                       label="3/2 <i>D</i><sub>&#8741;</sub>", label_offset=-0.06, font_size=28, arrow_indent=0.02
                       )
fig.add_annotation(
    x=freq[0], xref='x',
    y=0.985, yref='paper',
    text='𝜈<sub>34</sub>',
    showarrow=False,
    font_size=28,
)

fig.add_annotation(
    x=freq[1], xref='x',
    y=0.985, yref='paper',
    text='𝜈<sub>13</sub>',
    showarrow=False,
    font=dict(size=28),
)

fig.update_layout(yaxis_range=yrange, xaxis_range=xrange)

fig.show(renderer='browser')
# fig.write_image('../Figures/Manuscript/TF_simulations/NMR_dipolar_ZFS.pdf')


#### Only isotropic hyperfine coupling of A_iso = 2 MHz

In [ ]:
xarr = xr.load_dataset('ALC_simulations/Data/TF_NMR_isotropic_cc.nc')
# print(ds)
theta = 0
B = 0

freq = xarr['frequencies'].sel(B=B, theta=theta).values * 1e3 # Convert to GHz
amps = xarr['amplitudes'].sel(B=B, theta=theta).values

# Define the frequency axis for plotting
freq_min, freq_max = -2.5, 2.5
n_points = 8000
freq_axis = np.linspace(freq_min, freq_max, n_points)
gamma = 0.01  # MHz

xrange = [freq_min, freq_max]
yrange = [-0.032, 1.2]

spectrum = get_concoluted_spectrum(freq, amps, freq_axis, gamma)

# Plot
fig = create_ZF_base_figure()
fig.add_trace(go.Scatter(x=freq_axis, y=spectrum, mode='lines', line=dict(color='black', width=2)))

fig.add_annotation(
    x=freq[0], xref='x',
    y=0.995, yref='paper',
    text='𝜈<sub>24</sub>',
    showarrow=False,
    font_size=28,
)
fig.add_annotation(
    x=freq[0], xref='x',
    y=0.93, yref='paper',
    text='𝜈<sub>12</sub>',
    showarrow=False,
    font_size=28,
)
fig.update_layout(yaxis_range=yrange, xaxis_range=xrange)

fig.show(renderer='browser')
# fig.write_image('../Figures/Manuscript/TF_simulations/NMR_iso_ZFS.pdf')


### muSR

#### Only dipolar coupling of D_parallel = 2 MHz
expect 4 peaks with 1/2 D splitting, 2 peaks in middle are SQE transitions which are invisible in NMR

In [ ]:
xarr = xr.load_dataset('ALC_simulations/Data/TF_muSR_dipolar_cc.nc')
# print(ds)
theta = 0
B = 0

freq = xarr['frequencies'].sel(B=B, theta=theta).values * 1e3 # Convert to GHz
amps = xarr['amplitudes'].sel(B=B, theta=theta).values

# Define the frequency axis for plotting
freq_min, freq_max = -2.5, 2.5
n_points = 8000
freq_axis = np.linspace(freq_min, freq_max, n_points)
gamma = 0.01  # MHz

xrange = [freq_min, freq_max]
yrange = [-0.00375, 0.14]

spectrum = get_concoluted_spectrum(freq, amps, freq_axis, gamma)
# Plot
fig = create_ZF_base_figure()
fig.add_trace(go.Scatter(x=freq_axis, y=spectrum, mode='lines', line=dict(color='black', width=2), showlegend=False))
add_doubleheaded_arrow(fig, xrange, yrange, x1=freq[0], x2=freq[2], y=0.75,
                       label="1/2 <i>D</i><sub>&#8741;</sub>", label_offset=-0.06, font_size=28, arrow_indent=0.02
                       )

shift_to_center = 0.009 * abs(xrange[1] - xrange[0])

fig.add_annotation(
    x=freq[1], xref='x',
    y=0.985, yref='paper',
    text='𝜈<sub>34</sub>',
    showarrow=False,
    font_size=28,
)
fig.add_annotation(
    x=freq[0], xref='x',
    y=0.985, yref='paper',
    text='𝜈<sub>24</sub>',
    showarrow=False,
    font_size=28,
)
fig.add_annotation(
    x=freq[2], xref='x',
    y=0.985, yref='paper',
    text='𝜈<sub>12</sub>',
    showarrow=False,
    font_size=28,
)
fig.add_annotation(
    x=freq[3], xref='x',
    y=0.985, yref='paper',
    text='𝜈<sub>13</sub>',
    showarrow=False,
    font_size=28,
)

fig.update_layout(yaxis_range=yrange, xaxis_range=xrange)
fig.show(renderer='browser')
# fig.write_image('../Figures/Manuscript/TF_simulations/muSR_dipolar_ZFS.pdf')


#### Only isotropic hyperfine coupling of A_iso = 2 MHz

3 peaks with A splitting

In [ ]:
xarr = xr.load_dataset('ALC_simulations/Data/TF_muSR_isotropic_cc.nc')
# print(ds)
theta = 0
B = 0

freq = xarr['frequencies'].sel(B=B, theta=theta).values * 1e3 # Convert to GHz
amps = xarr['amplitudes'].sel(B=B, theta=theta).values

# Define the frequency axis for plotting
freq_min, freq_max = -2.5, 2.5
n_points = 8000
freq_axis = np.linspace(freq_min, freq_max, n_points)
gamma = 0.01  # MHz

xrange = [freq_min, freq_max]
yrange = [-0.008, 0.3]

spectrum = get_concoluted_spectrum(freq, amps, freq_axis, gamma)

# Plot
fig = create_ZF_base_figure()
fig.add_trace(go.Scatter(x=freq_axis, y=spectrum, mode='lines', line=dict(color='black', width=2), showlegend=False))
add_doubleheaded_arrow(fig, xrange, yrange, x1=freq[1], x2=freq[0], y=0.35,
                       label=r'$\LARGE A_\text{iso}$', label_offset=-0.06, font_size=20
                       )
add_doubleheaded_arrow(fig, xrange, yrange, x1=freq[2], x2=freq[3], y=0.35,
                       label=r'$\LARGE A_\text{iso}$', label_offset=-0.06, font_size=20
                       )
fig.add_annotation(
    x=freq[1], xref='x',
    y=0.48, yref='paper',
    text='𝜈<sub>34</sub>',
    showarrow=False,
    font_size=28,
)
fig.add_annotation(
    x=freq[0], xref='x',
    y=0.995, yref='paper',
    text='𝜈<sub>24</sub>',
    showarrow=False,
    font_size=28,
)
fig.add_annotation(
    x=freq[0], xref='x',
    y=0.93, yref='paper',
    text='𝜈<sub>12</sub>',
    showarrow=False,
    font_size=28,
)
fig.add_annotation(
    x=freq[3], xref='x',
    y=0.48, yref='paper',
    text='𝜈<sub>13</sub>',
    showarrow=False,
    font_size=28,
)

fig.update_layout(yaxis_range=yrange, xaxis_range=xrange)

fig.show(renderer='browser')
# fig.write_image('../Figures/Manuscript/TF_simulations/muSR_iso_ZFS.pdf')


### EPR zero field splitting


In [ ]:
# xarr = xr.load_dataset('Data/TF_EPR_dipolar_cc.nc')
# # print(ds)
# theta = 0
# B = 0
#
# freq = xarr['frequencies'].sel(B=B, theta=theta).values * 1e3 # Convert to GHz
# amps = xarr['amplitudes'].sel(B=B, theta=theta).values
#
# # Define the frequency axis for plotting
# freq_min, freq_max = -2, 2
# n_points = 8000
# freq_axis = np.linspace(freq_min, freq_max, n_points)
# gamma = 0.01  # MHz
#
# spectrum = get_concoluted_spectrum(freq, amps, freq_axis, gamma)
#
# # Plot
# fig = create_ZF_base_figure()
# fig.add_trace(go.Scatter(x=freq_axis, y=spectrum, mode='lines', line=dict(color='black', width=2)))
# add_doubleheaded_arrow(fig, freq[0], freq[2], 0.8, r'$\Large \frac{1}{2}D_\parallel$', label_offset=-2, font_size=20)
#
# fig.show(renderer='browser')
# # fig.write_image('../Figures/Manuscript/TF_simulations/EPR_dipolar_ZFS.pdf')


# Avoided level-crossing
## Analyitical model

In [ ]:
def muon_polarization_time_integrated(magnetic_fields, theta, Pz_0, off_resonance_state_rate, A_iso, D_parallel):
    """
    Calculate the time-integrated muon polarization for a given magnetic field and angle theta.
    """
    gamma_muon = 135.5 # in MHz/T
    inv_muon_lifetime = 0.4551 # in MHz
    osc_relaxation = inv_muon_lifetime + off_resonance_state_rate # in MHz

    def calc_q(theta, D_parallel):
        return 0.75 * D_parallel * np.sin(theta) * np.cos(theta)

    def calc_nu_mu(A_iso, D_parallel, theta):
        return (A_iso + D_parallel/2 * (3 * np.cos(theta)**2 - 1)) / 2

    q = calc_q(theta, D_parallel)
    nu_mu = calc_nu_mu(A_iso, D_parallel, theta)
    nu_mu0 = gamma_muon * magnetic_fields

    nominator = 0.5 * q**2 * Pz_0
    denominators = (osc_relaxation/2/np.pi)**2 + q**2 + (nu_mu0 - nu_mu)**2

    return 1 - nominator/denominators

magnetic_fields = np.linspace(1.85, 1.95, 3000) # in Tesla
thetas = np.radians([1, 5, 20, 45, 70, 85, 89]) # given in degrees and converted to radians
Pz_0 = 1.0
off_resonance_state_rate = 0
A_iso = 514.8 # in MHz
D_parallel = 2 # in MHz

df = pd.DataFrame({'B / T': magnetic_fields})
for theta in thetas:
    df[f"θ = {np.degrees(theta):.0f}°"] = muon_polarization_time_integrated(magnetic_fields, theta, Pz_0, off_resonance_state_rate, A_iso, D_parallel)

thetas_powder = np.radians(np.linspace(0, 90, 500))
polarization_matrix = np.column_stack([
    muon_polarization_time_integrated(magnetic_fields, theta, Pz_0, off_resonance_state_rate, A_iso, D_parallel)
    for theta in thetas_powder
])

df_powder = pd.DataFrame(polarization_matrix, columns=[f"θ_{i}" for i in range(len(thetas_powder))])
df_powder.insert(0, "B", magnetic_fields)

# weighted mean across thetas
weights = np.sin(thetas_powder)
polarizations_weighted = np.average(polarization_matrix, axis=1, weights=weights)

df["Powder"] = polarizations_weighted

# Color handling + plotting
# TODO: decide on best color scheme for different angles
pastel = pc.qualitative.Set2
colors = pastel[:len(thetas)] + ["black"]  # auto-adjust to number of angles + powder avg

fig = px.line(df, x="B / T", y=df.columns[1:], color_discrete_sequence=colors)
fig.update_layout(height=400, width=600,
                  xaxis_title='<i>B</i> / T',
                  xaxis_range=[1.875, 1.935],
                  # yaxis_title=r"$\Large \left < P_z \right >$",  # _z is a bit ugly like this
                  yaxis_title='<i>P<sub>z</sub></i>',
                  legend=dict(y=0.015, yanchor="bottom", title=''),
                  )
# fig.write_image('../Figures/Manuscript/ALC/ALC_analytical.pdf')
fig.show()

### Breit-Rabi of numerical model

In [ ]:
data = loadmat('../MatLab/ALCvsMQ/Data/ALC_breit_rabi_diagram.mat')

magnetic_fields = data['magnetic_fields']
energy_levels = pd.DataFrame(data['E_matrix_3D'].transpose())

# Adjust ordering for plotting
energy_levels = energy_levels.iloc[:, ::-1]
energy_levels[0], energy_levels[1] = energy_levels[1], energy_levels[0]


fig = go.Figure()
for idx, col in enumerate(energy_levels.columns):
    fig.add_trace(go.Scatter(
        x=magnetic_fields.flatten(),
        y=energy_levels[col],
        mode="lines",
        showlegend=False,
        line=dict(dash="dot" if idx in [1, 3] else "solid"),
    ))

fig.add_vline(x=1.8919, line=dict(color='black', width=2, dash='dash'))

fig.update_layout(height=400, width=600,
                  xaxis_title='<i>B</i> / T',
                  yaxis_title='𝜈 / MHz',
                  legend=dict(y=0.985, yanchor="top", title='',
                              x=0.01, xanchor="left"),)

# Add state labels
labels = ['|1⟩', '|2⟩', '|3⟩', '|4⟩']
x_position = 2.25
y_positions = [198+30, 198-30, -200-30, -200+40]
colors = px.colors.qualitative.G10[:4]
for y, label, color in zip(y_positions, labels, colors):
    fig.add_annotation(
        x=x_position, y=y,
        text=label,
        showarrow=False,
        font=dict(size=24, color=color),
    )

# fig.write_image('../Figures/Manuscript/ALC/ALC_breit_rabi.pdf')
fig.show()

upper_diff = abs(energy_levels[3] - energy_levels[2])
lower_diff = abs(energy_levels[1] - energy_levels[0])

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=magnetic_fields.flatten(),
    y=upper_diff,
    mode="lines",
    name='𝜈<sub>12</sub>',
    line=dict(color=px.colors.qualitative.G10[4])
))

fig.add_trace(go.Scatter(
    x=magnetic_fields.flatten(),
    y=lower_diff,
    mode="lines",
    name='𝜈<sub>34</sub>',
    line=dict(color=px.colors.qualitative.G10[5])
))

fig.add_vline(x=1.8919, line=dict(color='black', width=2, dash='dash'))

fig.update_layout(height=400, width=600,
                  xaxis_title='<i>B</i> / T',
                  yaxis_title='Δ𝜈 / MHz',
                  legend=dict(y=0.985, yanchor="top", title='',
                              x=0.01, xanchor="left"),
                  yaxis_range=[0, 4],
                  yaxis_tickvals=[0, 1, 2, 3, 4],)

# fig.write_image('../Figures/Manuscript/ALC/ALC_breit_rabi_deltaE.pdf')
fig.show()


## Numerical model using Spidyan

In [ ]:
num_data_angles = loadmat('../MatLab/ALCvsMQ/Data/num_ALC_simulation_thetas.mat')
num_data_powder = loadmat('../MatLab/ALCvsMQ/Data/num_ALC_simulation_powder.mat')
thetas = [1, 5, 20, 45, 70, 85, 89]  # thetas of the single orientation spectra

spectra = num_data_angles['spectra'][:, 0]  # shape (n_B, n_thetas) (0 is for O = Ix)

col_names = [f"θ = {theta}°" for theta in thetas]
df = pd.DataFrame(spectra.transpose(), columns=col_names)
magnetic_fields = num_data_angles['magnetic_fields'].flatten()  # shape (n_B,)

# powder average
# df['Powder'] = num_data_powder['powder_spectrum']

pastel = pc.qualitative.Set2
colors = pastel[:len(thetas)] + ["black"]  # auto-adjust to number of angles + powder avg

fig = go.Figure()

# Use normal graph_objects as the dashed line is not added in the foreground for some stupid reason
for col, color in zip(df.columns, colors):
    fig.add_trace(go.Scatter(
        x=magnetic_fields,
        y=df[col],
        mode="lines",
        line=dict(color=color),
        name=col
    ))
fig.add_trace(go.Scatter(x=num_data_powder['magnetic_fields'].flatten(),
                         y=num_data_powder['powder_spectrum'].flatten(),
                         mode='lines',
                         line=dict(color='red', width=2, dash='dash'),
                         name='Powder'))

fig.update_layout(height=400, width=600,
                  xaxis_title='<i>B</i> / T',
                  xaxis_range=[1.865, 1.925],
                  # yaxis_title=r"$\Large \left < P_z \right >$",  # _z is a bit ugly like this
                  yaxis_title='<i>P<sub>z</sub></i>',
                  legend=dict(y=0.015, yanchor="bottom", title=''),
                  )
fig.show()
# fig.write_image('../Figures/Manuscript/ALC/ALC_numerical.pdf')

#### Comparison of numerical and analytical powder average

In [ ]:
x_num = num_data_powder['magnetic_fields'].flatten()
y_num = num_data_powder['powder_spectrum'].flatten()

magnetic_fields = x_num
Pz_0 = 1.0
off_resonance_state_rate = 0
A_iso = 514.8 # in MHz
D_parallel = 2 # in MHz

thetas_powder = np.radians(np.linspace(0, 90, 500))
polarization_matrix = np.column_stack([
    muon_polarization_time_integrated(magnetic_fields, theta, Pz_0, off_resonance_state_rate, A_iso, D_parallel)
    for theta in thetas_powder
])

df_powder = pd.DataFrame(polarization_matrix, columns=[f"θ_{i}" for i in range(len(thetas_powder))])
df_powder.insert(0, "B", magnetic_fields)

# weighted mean across thetas
weights = np.sin(thetas_powder)
y_ana = np.average(polarization_matrix, axis=1, weights=weights)

x_ana = magnetic_fields

shift_peak = x_ana[y_ana.argmin()] - x_num[y_num.argmin()]
print(f'Shift of peaks: {shift_peak}')

x_num_shifted = x_num + shift_peak

fig = go.Figure()
# Analytical
fig.add_trace(go.Scatter(x=x_ana, y=y_ana,
                         mode='lines',
                         line=dict(color='black', width=2),
                         name='Analytical'))
# Numerical
fig.add_trace(go.Scatter(x=x_num_shifted, y=y_num,
                         mode='lines',
                         line=dict(color='red', width=2, dash='dash'),
                         name='Numerical'))
fig.update_layout(height=400, width=600,
                  xaxis_title='<i>B</i> / T',
                  xaxis_range=[1.87, 1.93],
                  yaxis_title='<i>P<sub>z</sub></i>',
                  yaxis=dict(
                      tick0=0.7,
                      dtick=0.1,
                      range=[0.68, 1],
                  ),
                  legend=dict(y=0.015, yanchor="bottom", title=''),
                  )
# fig.write_image('../Figures/Manuscript/ALC/ALC_analytical_vs_numerical_powder_average.pdf')
fig.show()

Shift between numerical and analytical model for each theta

In [ ]:
df = pd.read_csv('../MatLab/ALCvsMQ/ana_vs_num_peak_positions.csv')
print(np.mean(df['peak_pos_diff'])*1000)  # in mT

fig = go.Figure()
fig.add_trace(go.Scatter(x=df['theta'], y=df['peak_pos_diff']*1000,  # convert to mT
                         mode='lines',
                         line=dict(color='black', width=2),))
fig.update_layout(height=400, width=600,
                  xaxis_title='θ / °',
                  yaxis_title='Δ<i>B</i> / mT',
                  # xaxis_range=[1.87, 1.93],
                  )
# fig.write_image('../Figures/Manuscript/ALC/ALC_analytical_vs_numerical_peak_position_difference.pdf')
fig.show()

## Limit of analytical model

In [ ]:
# All data simulated using T=15.5 MHz, theta=45
import glob
import re

path = '../MatLab/ALCvsMQ/Data/num_ALC_simulation_limit*.mat'

def extract_number(filename):
    match = re.search(r'_A(\d+)\.mat$', filename)
    return int(match.group(1)) if match else float('inf')

files = sorted(glob.glob(path), key=extract_number)

df_num = pd.DataFrame()
for idx, file in enumerate(files):
    print(file)
    mat = loadmat(file)
    df_num[idx] = mat['spectra'][0, 0, :]

print(df_num)

df_num.index = mat['magnetic_fields'].flatten()

magnetic_fields = np.linspace(0, 0.4, 3000)
Pz_0 = 1.0
off_resonance_state_rate = 0
A_iso_list = [3, 10, 50] # in MHz
D_parallel = 15.5 # in MHz
theta = np.radians(45)

polarization_matrix = np.column_stack([
    muon_polarization_time_integrated(magnetic_fields, theta, Pz_0, off_resonance_state_rate, A_iso, D_parallel)
    for A_iso in A_iso_list
])

df_ana = pd.DataFrame(polarization_matrix, columns=[f"A_iso={A_iso}" for A_iso in A_iso_list])
df_ana.insert(0, "B", magnetic_fields)

print(df_ana)
# px.line(df_ana, x="B", y=df_ana.columns[1:5]).show()

colors = px.colors.qualitative.G10

fig = go.Figure()

# Plot numerical spectra (solid)
for i, col in enumerate(df_num.columns):
    color = colors[i % len(colors)]
    fig.add_trace(go.Scatter(
        x=df_num.index,
        y=df_num[col],
        mode='lines',
        line=dict(color=color, dash='solid'),
        name=f"{A_iso_list[i]}"
    ))

# Plot analytical spectra (dashed, same colors)
for i, col in enumerate(df_ana.columns[1:]):
    color = colors[i % len(colors)]
    fig.add_trace(go.Scatter(
        x=df_ana['B'],
        y=df_ana[col],
        mode='lines',
        line=dict(color=color, dash='dash'),
        showlegend=False,
    ))

fig.update_layout(
    xaxis_title='B / T',
    yaxis_title='<i>P<sub>z</sub></i>',
    xaxis_range=[0, 0.3],
    height=400,
    width=600,
    legend=dict(y=0.015, yanchor="bottom", title=''),
)
# TODO: decide on how to handle the legend

fig.show(renderer='browser')

### Time traces for numerical model

In [ ]:
# awg.s_rate = 20, theta = 45 deg, A_iso = 514.8 MHz, D_parallel = 2 MHz, T1 = T2 = 2.2 us
# On resonance case (at B = 1.8922 T)
df_on = loadmat('../MatLab/ALCvsMQ/Data/ALC_signal_time_evolution_on_resonance.mat')
print(df_on)
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_on['time_ds'].flatten()/1000,  # convert to microseconds))
                         y=np.real(df_on['trace'].flatten()),
                         mode='lines',
                         line=dict(color='black', width=2),
                         ))
fig.update_layout(height=400, width=600,
                  xaxis_range=[0, 12],
                  yaxis_title='<i>P<sub>z</sub></i>',
                  xaxis_title='<i>t</i> / μs',
                  margin=dict(b=80),
                  xaxis=dict(
                      title_standoff=8,
                  ),
                  yaxis=dict(
                      tick0=0.2,
                      dtick=0.2,
                  )
                 )
# fig.write_image('../Figures/Manuscript/ALC/numerical_ALC_time_trace_on_resonance.pdf')
fig.show()

# Off resonance case (at B = 1.82 T)
df_off = loadmat('../MatLab/ALCvsMQ/Data/ALC_signal_time_evolution_off_resonance.mat')

fig = go.Figure()
fig.add_trace(go.Scatter(x=df_off['time_ds'].flatten()/1000,  # convert to microseconds
                         y=np.real(df_off['trace'].flatten()),
                         mode='lines',
                         line=dict(color='black', width=2),
                         ))
fig.update_layout(height=400, width=600,
                  xaxis_range=[0, 12],
                  xaxis_title='<i>t</i> / μs',
                  margin=dict(b=80),
                  xaxis=dict(
                      title_standoff=8,
                  ),
                  yaxis_title='<i>P<sub>z</sub></i>',
                  yaxis=dict(
                      tick0=0.994,
                      dtick=0.002,
                  )
                 )
# fig.write_image('../Figures/Manuscript/ALC/numerical_ALC_time_trace_off_resonance.pdf')
fig.show()

## Repolarization
Plot of muon repolarization using simple analytical model

In [ ]:
def muon_repolarization(magnetic_fields, A_iso, gamma_electron=-28024.95, gamma_muon=135.5):
    B = A_iso / (gamma_electron + gamma_muon)
    temps = [(magnetic_field / B)**2 for magnetic_field in magnetic_fields]
    return [(1+2*temp)/(2*(1+temp)) for temp in temps]

magnetic_fields = np.linspace(0, 1, 2000) # in Tesla
A_iso = 514.8 # in MHz

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=np.log10(magnetic_fields),
    # x=magnetic_fields,
    y=muon_repolarization(magnetic_fields, A_iso),
    mode='lines',
    line=dict(color='black', width=2),
))
fig.update_layout(
    height=400, width=600,
    xaxis_title='log<sub>10</sub>(<i>B</i> / T)',
    xaxis=dict(
        title_standoff=8,
    ),
    yaxis_title='<i>P<sub>z</sub></i>',
    margin=dict(l=90, r=20, t=20, b=80),
    font_size=22,
)
fig.show()
# fig.write_image('../Figures/Manuscript/ALC/muon_repolarization_analytical.pdf')

# Pake Pattern for powder sample


# Energy level splitting as a function of B

## A_iso = 1 GHz, D_parallel = 0

In [ ]:
df = pd.read_csv('ALC_simulations/Data/energy_levels_A_iso_1GHz.csv', index_col=0)

df = df.iloc[:, ::-1]

magnetic_fields = df.index.values
fig = go.Figure()
for col in df.columns:
    fig.add_trace(go.Scatter(x=magnetic_fields, y=df[col],
                             mode='lines',
                             line=dict(width=2),
                             showlegend=False,
                             ))
fig.update_layout(height=400, width=600,
                  xaxis_title='<i>B</i><sub>0</sub>',
                  yaxis_title=r'$\Large E \; / \; (h A_\text{iso})$'
                  # yaxis_title='<i>E / A</i><sub>iso</sub>'
                 )

xrange = [0, 0.075]
yrange = [-1.7, 1.65]

# Add state labels
labels = ['|1⟩', '|2⟩', '|3⟩', '|4⟩']
print(reversed(labels))
x_position = df.index[120]  # in T
y_at_x = df.loc[x_position].values
print(y_at_x)
y_positions = [1.22508227+0.2, 0.85424302-0.16, -1.35424302-0.15, -0.72508227+0.2]
colors = px.colors.qualitative.G10[:4]
for y, label, color in zip(y_positions, labels, colors):
    fig.add_annotation(
        x=x_position, y=y,
        text=label,
        showarrow=False,
        font=dict(size=24, color=color),
    )

fig.update_layout(
    xaxis=dict(
        tickvals=[0],
        ticktext=['0'],
        showgrid=False,
        title_standoff=0,        # decrease distance between x label and axis
        minor=dict(ticks='', showgrid=False),  # remove minor ticks and minor grid
        range=xrange,
        ticklabelstandoff=-8,
    ),
    yaxis=dict(
        tickvals=[-1.5, -3/4, 1/4, 1, 1.5],
        ticktext=["−3/2", "−3/4", "1/4", "1", "3/2",],
        showgrid=False,
        title_standoff=8,        # decrease distance between x label and axis
        minor=dict(ticks='', showgrid=False),  # remove minor ticks and minor grid
        range=yrange,
    ),
    margin=dict(b=68),
)
add_doubleheaded_arrow(fig, xrange, yrange, x1=0.014, y=-0.75, y2=0.25, label=r'$\Large A_\text{iso}$', orientation='v', label_offset=0.03, arrow_indent=0.04, head_size=10)

# fig.write_image('../Figures/Manuscript/TF_simulations/energy_levels_Aiso_1GHz.pdf')
# fig.show(renderer='browser')

## A_iso = 0 GHz, D_parallel = 1 GHz, theta = 45°

In [ ]:
df = pd.read_csv('ALC_simulations/Data/energy_levels_A_iso_0GHz_D_1GHz_theta45.csv', index_col=0)

df = df.iloc[:, ::-1]

magnetic_fields = df.index.values
fig = go.Figure()
for col in df.columns:
    fig.add_trace(go.Scatter(x=magnetic_fields, y=df[col],
                             mode='lines',
                             line=dict(width=2),
                             showlegend=False,
                             ))
fig.update_layout(height=400, width=600,
                  xaxis_title='<i>B</i><sub>0</sub>',
                  yaxis_title=r'$\Large E \; / \; (h D_\parallel)$'
                  # yaxis_title='<i>E / A</i><sub>iso</sub>'
                 )

xrange = [0, 0.075]
yrange = [-1.7, 1.7]

# Add state labels
labels = ['|1⟩', '|2⟩', '|3⟩', '|4⟩']
print(reversed(labels))
x_position = df.index[120]  # in T
y_at_x = df.loc[x_position].values
print(y_at_x)
y_positions = [1.18263637+0.2, 0.83207356-0.16, -1.22503929-0.15, -0.78967064+0.2]
colors = px.colors.qualitative.G10[:4]
for y, label, color in zip(y_positions, labels, colors):
    fig.add_annotation(
        x=x_position, y=y,
        text=label,
        showarrow=False,
        font=dict(size=24, color=color),
    )

fig.update_layout(
    xaxis=dict(
        tickvals=[0],
        ticktext=['0'],
        showgrid=False,
        title_standoff=0,        # decrease distance between x label and axis
        minor=dict(ticks='', showgrid=False),  # remove minor ticks and minor grid
        range=xrange,
        ticklabelstandoff=-8,
    ),
    yaxis=dict(
        tickvals=[-1.5, -1, -0.5, 0, 1/4, 1, 1.5],
        ticktext=["−3/2", "−1", "−1/2", "0", "1/4", "1", "3/2"],
        showgrid=False,
        title_standoff=8,        # decrease distance between x label and axis
        minor=dict(ticks='', showgrid=False),  # remove minor ticks and minor grid
        range=yrange,
    ),
    margin=dict(b=68),
)
# add_doubleheaded_arrow(fig, xrange, yrange, x1=0.014, y=-0.75, y2=0.25, label=r'$\Large A_\text{iso}$', orientation='v', label_offset=0.03, arrow_indent=0.04, head_size=10)

# fig.write_image('../Figures/Manuscript/TF_simulations/energy_levels_D_1GHz_theta45.pdf')
fig.show(renderer='browser')

## A_iso = 0 GHz, D_parallel = 1 GHz, theta = 45°

In [ ]:
df = pd.read_csv('ALC_simulations/Data/energy_levels_A_iso_0GHz_D_1GHz_theta0.csv', index_col=0)

df = df.iloc[:, ::-1]

magnetic_fields = df.index.values
fig = go.Figure()
for col in df.columns:
    fig.add_trace(go.Scatter(x=magnetic_fields, y=df[col],
                             mode='lines',
                             line=dict(width=2),
                             showlegend=False,
                             ))
fig.update_layout(height=400, width=600,
                  xaxis_title='<i>B</i><sub>0</sub>',
                  yaxis_title=r'$\Large E \; / \; (h D_\parallel)$'
                  # yaxis_title='<i>E / A</i><sub>iso</sub>'
                 )

xrange = [0, 0.075]
yrange = [-1.7, 1.7]

# Add state labels
labels = ['|1⟩', '|2⟩', '|3⟩', '|4⟩']
print(reversed(labels))
x_position = df.index[120]  # in T
y_at_x = df.loc[x_position].values
print(y_at_x)
y_positions = [1.22508227+0.2, 0.76580148-0.16, -1.26580148-0.15, -0.72508227+0.2]
colors = px.colors.qualitative.G10[:4]
for y, label, color in zip(y_positions, labels, colors):
    fig.add_annotation(
        x=x_position, y=y,
        text=label,
        showarrow=False,
        font=dict(size=24, color=color),
    )

fig.update_layout(
    xaxis=dict(
        tickvals=[0],
        ticktext=['0'],
        showgrid=False,
        title_standoff=0,        # decrease distance between x label and axis
        minor=dict(ticks='', showgrid=False),  # remove minor ticks and minor grid
        range=xrange,
        ticklabelstandoff=-8,
    ),
    yaxis=dict(
        tickvals=[-1.5, -1, -0.5, 0, 1/4, 1, 1.5],
        ticktext=["−3/2", "−1", "−1/2", "0", "1/4", "1", "3/2"],
        showgrid=False,
        title_standoff=8,        # decrease distance between x label and axis
        minor=dict(ticks='', showgrid=False),  # remove minor ticks and minor grid
        range=yrange,
    ),
    margin=dict(b=68),
)
# add_doubleheaded_arrow(fig, xrange, yrange, x1=0.014, y=-0.75, y2=0.25, label=r'$\Large A_\text{iso}$', orientation='v', label_offset=0.03, arrow_indent=0.04, head_size=10)

# fig.write_image('../Figures/Manuscript/TF_simulations/energy_levels_D_1GHz_theta0.pdf')
# fig.show(renderer='browser')

# MQ Figures

## Numerical simulation of MQ spectra

### Rabi-Oscillations

In [ ]:
# On resonance case
df_on = loadmat('../MatLab/ALCvsMQ/Data/MQ_signal_time_evolution_on_resonance.mat')
print(df_on)
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_on['time_ds'].flatten()/1000,  # convert to microseconds))
                         y=np.real(df_on['trace'].flatten()),
                         mode='lines',
                         line=dict(color='black', width=2),
                         ))
fig.update_layout(height=400, width=600,
                  xaxis_range=[0, 12],
                  yaxis_title='<i>P<sub>z</sub></i>',
                  xaxis_title='<i>t</i> / μs',
                  margin=dict(b=80),
                  xaxis=dict(
                      title_standoff=8,
                  ),
                  yaxis=dict(
                      tick0=0.2,
                      dtick=0.2,
                  )
                 )
# fig.write_image('../Figures/Manuscript/MQ_simulations/MQ_time_trace_on_resonance.pdf')
fig.show()

# Off resonance case
df_on = loadmat('../MatLab/ALCvsMQ/Data/MQ_signal_time_evolution_off_resonance.mat')
print(df_on)
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_on['time_ds'].flatten()/1000,  # convert to microseconds))
                         y=np.real(df_on['trace'].flatten()),
                         mode='lines',
                         line=dict(color='black', width=2),
                         ))
fig.update_layout(height=400, width=600,
                  xaxis_range=[0, 12],
                  yaxis_title='<i>P<sub>z</sub></i>',
                  xaxis_title='<i>t</i> / μs',
                  margin=dict(b=80),
                  xaxis=dict(
                      title_standoff=8,
                  ),
                  yaxis=dict(
                      tick0=0.2,
                      dtick=0.2,
                  )
                 )
# fig.write_image('../Figures/Manuscript/MQ_simulations/MQ_time_trace_off_resonance.pdf')
fig.show()


## Angle dependence of MQ spectra

System used: A_iso = 0 MHz, D_parallel = 15.5 MHz


In [ ]:
df = loadmat('../MatLab/ALCvsMQ/Data/MQ_pcolor_data_long_run.mat')

magnetic_fields = df['magnetic_fields'].flatten()*1000  # convert to mT
thetas = np.degrees(df['thetas'].flatten())
spectra = df['spectra']  # shape (n_B, n_thetas)

det_op = 0  # 0 for Iz, 1 for Ix, 2 for Sz

fig = go.Figure(data=go.Contour(
    z=spectra[:, det_op],
    x=magnetic_fields,
    y=thetas,
    colorscale='Viridis',
    colorbar=dict(title='<i>P<sub>z</sub></i>'),
    contours=dict(
        start=0,
        end=1,
        size=0.05,
    )
))

fig.update_layout(
    height=400, width=700,
    xaxis_title='<i>B</i> / mT',
    yaxis_title='θ / °',
    yaxis_tickvals=np.arange(0, 91, 15),
    margin=dict(b=80),
    xaxis_range=[81.25, 83.15],
)

# fig.write_image('../Figures/Manuscript/MQ_simulations/MQ_angle_dependence_contour.pdf')
fig.show()

print(len(magnetic_fields))

## Angle dependence of SQ electron spectra

System used: A_iso = 0 MHz, D_parallel = 15.5 MHz

In [ ]:
df = loadmat('../MatLab/ALCvsMQ/Data/SQ_pcolor_electron_large_run.mat')

magnetic_fields = df['magnetic_fields'].flatten()
thetas = np.degrees(df['thetas'].flatten())
spectra = df['spectra']  # shape (n_B, n_thetas)

det_op = 2  # 0 for Iz, 1 for Ix, 2 for Sz

fig = go.Figure(data=go.Contour(
    z=spectra[:, det_op],
    x=magnetic_fields*1000,
    y=thetas,
    colorscale='Viridis',
    colorbar=dict(title='<i>P<sub>z</sub></i>'),
    contours=dict(
        start=0,
        end=1,
        size=0.05,
    )
))

fig.update_layout(
    height=400, width=700,
    xaxis_title='<i>B</i> / mT',
    yaxis_title='θ / °',
    yaxis_tickvals=np.arange(0, 91, 15),
    margin=dict(b=80),
    # xaxis_range=[0.081, 0.0834],
    xaxis_range=[81.25, 83.15],
)

# fig.write_image('../Figures/Manuscript/MQ_simulations/electron_spectrum_angle_dependence.pdf')
fig.show()

## $A_\text{iso}$ dependence of MQ spectra

$D_\parallel$ = 15.5 MHz, powder spectra

In [ ]:
df = loadmat('../MatLab/ALCvsMQ/Data/num_MQ_powder_spectra_D15_5MHz.mat')

A_isos = df['A_isos'].flatten()*1000  # convert to MHz
magnetic_fields = df['magnetic_fields'].flatten()*1000  # convert to mT

print(df['powder_spectra'][0, 0].flatten())

powder_spectra = pd.DataFrame()

for idx, A_iso in enumerate(A_isos):
    powder_spectra[A_iso] = df['powder_spectra'][0, idx].flatten()

fig = go.Figure()
for A_iso in A_isos:
    fig.add_trace(go.Scatter(
        x=magnetic_fields,
        y=powder_spectra[A_iso],
        mode='lines',
        name=f'{A_iso:.0f} MHz',
    ))

fig.update_layout(
    height=400, width=600,
    xaxis_title='<i>B</i> / mT',
    yaxis_title='<i>P<sub>z</sub></i>',
)

fig.show()

## $T_2$ dependence of MQ spectra

$D_\parallel$ = 15.5 MHz, $A_\text{iso}$=1.4 MHz, powder spectra

In [ ]:
df = loadmat('../MatLab/ALCvsMQ/Data/num_MQ_powder_spectra_D15_5MHz_different_T2.mat')

T2_list = df['T2_array'].flatten()  # in MHz
magnetic_fields = df['magnetic_fields'].flatten()*1000  # convert to mT

print(df['powder_spectra'][0, 0].flatten())

powder_spectra = pd.DataFrame()

for idx, A_iso in enumerate(T2_list):
    powder_spectra[A_iso] = df['powder_spectra'][0, idx].flatten()

fig = go.Figure()
for T2 in T2_list:
    fig.add_trace(go.Scatter(
        x=magnetic_fields,
        y=powder_spectra[T2],
        mode='lines',
        name=f'{T2} ns',
    ))

fig.update_layout(
    height=400, width=600,
    xaxis_title='<i>B</i> / mT',
    yaxis_title='<i>P<sub>z</sub></i>',
)

fig.show()